# Neural Networks

In [7]:
import numpy as np
import pandas as pd
import statistics
import random
import scipy

## A Simple example

In [8]:
class Neuron:
    def __init__(self, weights, activation_func, bias = 0):
        self.weights = weights
        self.activation = activation_func
        self.bias = bias
    def output(self, x):
        total = np.dot(self.weights, x)
        return self.activation(total + self.bias)

In [9]:
# Activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [10]:
n = Neuron([0.2, 0.3, 0.4], sigmoid)
# Inputs
inp = np.array([2,1,7])
result = n.output(inp)
print(result)

0.9706877692486436


The purpose of training is to calculate (find) appropriate values for weights which is the outcome of the training.

## Layers

- Input layer: Each input is represented as a node. However, they do NOT act as a node. The input and output value is same and equal. # of nodes = # features (columns)
- Hidden layer(s): # layers and nodes are dependent on the complexity of the problem.
    - If the problem is lineary separable: No layers needed.
    - One layer is usually adequate and two layers are usually capable of solving most of the problems
    - Number of nodes: ??? 
- Output layer: # of nodes depends on the problem we're trying to solve.

### Datasets in Supervised Neural Networks